# 2번 과제 TOPSIS 기법 프로세스
**TOPSIS 기법 : 요구사항 우선순위기법(technique for order of preference by similarity to ideal solution:
TOPSIS)은 평가요소(Criteria)가 여러 가지이고 단위가 다른 경우와 대안(Alternatives)들이 여러 가
지인 경우에 적합한 의사결정 방법이다. 요구사항 우선순위 기법은 가장 이상적인 해에 어떤 대안
들이 가까운지를 보여주며 대안들의 점수를 보여주기 때문에 순위를 매길 수 있는 의사결정 방법
이며 정성적인 방법과 정량적인 방법을 결합한 형태로 많은 문제에 적용되어왔다[김기태, 배송채널 선정문제에 대한 TOPSIS 응용, 2017]**

## TOPSIS기법을 사용하기 위해선 각 변수(혹은 컬럼)의 가중치가 필요한데 이 가중치를 구하기 위해서 다중회귀분석 계수를 사용하고자 한다.
 1. 4510개의 격자의 중심점을 좌표로 변환하여 데이터프레임을 구축한다.
 2.  어린이 교통사고에 영향을 미칠 수 있는 변수(컬럼)을 4510개의 중심점 좌표와의 거리를 계산하여 반경 300m(법적 어린이보호구역)내에 있으면 각 격자에 할당해준다.
 3. 사고횟수를 종속변수로 설정, 나머지 변수들을 독립변수로 하여 정규화를 진행 후 다중회귀 분석을 진행하였고, p-value가 0.1이하인 변수와 다중공선성 문제(VIF 기준 7이상)을 발생시키는 변수를 제거하여 새로운 데이터프레임을 구축하였다.  
 4. 각 변수의 회귀계수를 정규화(Coefficient를 절댓값화 후 이들의 합으로 나눠줌) 시켜주어 가중치로 결정하였다.

## 91개의 후보군(어린이보호구역)에 대해 TOPSIS기법을 적용한다.
 5. 앞선 과정과 동일하게 91개의 후보군(기존 어린이보호구역)에 대하여 같은 절차를 반복한다.
 6. 각 컬럼들을 정규화를 해주어야 하는데, 다중회귀계수가 양이면 이익요소, 음이면 비용요소로 구분하였고 비례점수법과 벡터정규화법을 통해 정규화를 진행해주었다. 
 7. 가중치와 정규화된 값들을 곱해 새로운 데이터프레임을 구축한다.
 8. 각 컬럼별 최대값의 집합을 "최선의 해(IS)", 최소값의 집합을 "최악의 해(NIS)"로 명시하고, 괴리도와 상대적 근접도를 통해 각 Instance의 IS와 NIS까지 거리를 산출한다.
 9. IS와 가까울수록 최선의 해와 가까우며, 이는 곧 시설물이 우선적으로 설치가 필요하다고 판단하여 IS의 상위 20개 도출

# 데이터 읽어오기
- osan_accident.xlsx : TAAS에서 추출한 오산시 교통사고 데이터 입니다.

In [ ]:
import pandas as pd
import json
from math import sin, cos, sqrt, atan2, radians
import pyproj
from shapely.geometry import Point, Polygon
from functools import partial
from shapely.ops import transform
import geopandas as gpd
import math

osan_accident = pd.read_excel('osan_accident.xlsx') 
osan_accident = osan_accident[['사고번호','좌표','사고유형','법규위반','사망자수']]
df1 = pd.read_csv('1.오산시_주정차단속(2018~2020).csv')
df2 = json.load(open('2.오산시_어린이교통사고_격자.geojson', encoding = 'utf-8'))
df3 = json.load(open('3.오산시_차량등록현황_격자.geojson', encoding = 'utf-8'))
df4 = json.load(open('4.오산시_연령별_거주인구격자(총인구).geojson', encoding = 'utf-8'))
df5 = json.load(open('5.오산시_연령별_거주인구격자(유소년).geojson', encoding = 'utf-8'))
df6 = json.load(open('6.오산시_연령별_거주인구격자(생산가능인구).geojson', encoding = 'utf-8'))
df7 = json.load(open('7.오산시_연령별_거주인구격자(고령).geojson', encoding = 'utf-8'))
df8 = pd.read_csv('8.오산시_유동인구(2019).csv')
df9 = pd.read_csv('9.오산시_어린이보호구역.csv')
df10 = pd.read_csv('10.오산시_학교위치정보.csv')
df11 = json.load(open('11.오산시_초등학교_통학구.geojson', encoding = 'utf-8'))
df12 = json.load(open('12.오산시_중학교_학군.geojson', encoding = 'utf-8'))
df13 = pd.read_csv('13.오산시_어린이집_유치원현황.csv')
df14 = pd.read_csv('14.오산시_기상데이터(2010~2019).csv')
df15 = pd.read_csv('15.오산시_무인교통단속카메라.csv')
df16 = pd.read_csv('16.오산시_도로안전표지표준데이터.csv')
df17 = json.load(open('17.오산시_횡단보도.geojson', encoding = 'utf-8'))
df18 = pd.read_csv('18.오산시_과속방지턱표준데이터.csv')
df19 = json.load(open('19.오산시_신호등.geojson', encoding = 'utf-8'))
df20 = pd.read_csv('20.오산시_CCTV설치현황.csv')
df21 = json.load(open('21.오산시_인도.geojson', encoding = 'utf-8'))
df22 = pd.read_csv('22.오산시_버스정류장.csv')
df23 = json.load(open('23.오산시_상세도로망_LV6.geojson', encoding = 'utf-8'))
df24 = pd.read_csv('24.평일_전일,시간대별_오산시_추정교통량_Level6.csv')
df25 = pd.read_csv('25.평일_전일_오산시_혼잡빈도강도_Level6.csv')
df26 = pd.read_csv('26.평일_전일_오산시_혼잡시간강도_Level6.csv')
df27 = json.load(open('27.오산시_도로명주소_건물.geojson', encoding = 'utf-8'))
df28 = json.load(open('28.오산시_건물연면적_격자.geojson', encoding = 'utf-8'))
df29 = pd.read_csv('29.오산시_체육시설현황.csv')
df30 = pd.read_csv('30.오산시_학원 및 교습소 현황.csv')
df31 = json.load(open('31.오산시_법정경계(시군구).geojson', encoding = 'utf-8'))
df32 = json.load(open('32.오산시_행정경계(읍면동).geojson', encoding = 'utf-8'))
df33 = json.load(open('33.오산시_법정경계(읍면동).geojson', encoding = 'utf-8'))
df34 = json.load(open('34.오산시_지적도.geojson', encoding = 'utf-8'))

## 데이터 전처리
- 1) 4510개의 격자의 중심점 좌표 데이터프레임을 구축한다.
- 2) 1번 과제 솔루션과 같은 변수들을 추출하되, 법적 어린이보호구역 반경 300m를 기준으로 각종 시설물을 할당하여준다.
- 3) 할당해주고자 하는 컬럼은 다음과 같다.
>주정차단속건수 ,
사고횟수(TAAS 데이터), 
차량개수,
유소년인구수,
생산가능인구수,
고령인구수,
어린이보호구역,
유치원/어린이집개수,
무인교통단속카메라개수,
도로안전표지판개수,
횡단보도개수,
과속방지턱개수,
신호등개수,
버스정류장개수,
건물개수,
건물연면적,
체육시설개수,
학원교습소개수,
초등학교개수,
어린이보호구역CCTV개수,
유동인구,
전체 추정교통량,
혼잡빈도강도,
혼잡시간강도,


## 1) 4510개의 격자의 중심점을 좌표로 변환하여 데이터프레임을 구축한다.

In [ ]:
#4510 개의 격자 중심점 좌표 데이터프레임을 구축한다.
df4['features'][0]['properties']['gid']
square_df = pd.DataFrame()
for i in range(len(df4['features'])):
    coord = df4['features'][i]['geometry']['coordinates'][0][0]
    n_dict = {'gid' : df4['features'][i]['properties']['gid'], 'center_lon':(coord[1][0] + coord[2][0])/2, 'center_lat' : (coord[0][1] + coord[1][1])/2}
    square_df = square_df.append(n_dict,ignore_index=True)
df = square_df.set_index('gid')
df

## 2) 어린이 교통사고에 영향을 미칠 수 있는 변수(컬럼)을 4510개의 중심점 좌표와의 거리를 계산하여 반경 300m(법적 어린이보호구역)내에 있으면 각 격자에 할당해준다.
- 시설물을 할당하여주는데에 시간을 줄이기 위해 컬럼명을 재정의해주고 중복된 좌표를 Groupby(sum)하여준다.

In [ ]:
#데이터프레임 컬럼명 재정의
df1 = df1.rename(columns={'단속위치_경도':'lon','단속위치_위도':'lat'}) #주정차단속
df9 = df9.rename(columns={'보호구역_경도':'lon','보호구역_위도':'lat'}) #어린이보호구역
df10 = df10.rename(columns={'학교위치_경도':'lon','학교위치_위도':'lat'}) #학교
df13 = df13.rename(columns={'시설위치_경도':'lon','시설위치_위도':'lat'}) #유치원
df15 = df15.rename(columns={'설치위치_경도':'lon','설치위치_위도':'lat'}) #무인교통단속카메라
df18 = df18.rename(columns={'설치위치_경도':'lon','설치위치_위도':'lat'}) #과속방지턱
df22 = df22.rename(columns={'정류장 위치_경도':'lon','정류장 위치_위도':'lat'}) #버스정류장
df29 = df29.rename(columns={'설치위치_경도':'lon','설치위치_위도':'lat'}) #체육시설
df30 = df30.rename(columns={'시설위치_경도':'lon','시설위치_위도':'lat'}) #학원 및 교습소

#학교 데이터에서 초등학교만 추출
df10 = df10[df10['학교구분']=='초등학교']

#코드 실행시간 단축을 위해 중복된 좌표를 Groupby sum() : 행을 축소하여 뒤에서 진행될 연산 과정을 줄일 수 있다.
df1['count'] = 1
df1 = df1.groupby(['lat','lon']).sum().reset_index()
df30['count'] = 1
df30 = df30.groupby(['lat','lon']).sum().reset_index()
df9 = df9.rename(columns={'CCTV설치대수' : 'count'})

#유동인구 데이터 전처리
# 어린이가 많이 다니는 14시 ~ 20시로 한정
df8_1 = df8.loc[:,['lon','lat','TMST_14','TMST_15','TMST_16','TMST_17','TMST_18','TMST_19','TMST_20']]
df8_1['mean'] = (df8_1['TMST_14'] + df8_1['TMST_15'] + df8_1['TMST_16'] + df8_1['TMST_17'] + df8_1['TMST_18'] + df8_1['TMST_19'] +df8_1['TMST_20'])/7
df8_1 = df8_1.loc[:,['lon','lat','mean']]

# 위도와 경도기준 groupby
df8_2 = df8_1.groupby(['lon','lat']).sum()
df8_2 = df8_2.reset_index()
df8_2 = df8_2.reindex(columns=['lat','lon','mean'])
df8_2.rename(columns={'mean':'count'},inplace=True)  #함수에 사용하기 위해 mean을 count로 대체. 

## 2) 어린이 교통사고에 영향을 미칠 수 있는 변수(컬럼)을 4510개의 중심점 좌표와의 거리를 계산하여 반경 300m(법적 어린이보호구역)내에 있으면 각 격자에 할당해준다.
- 코드의 재활용성을 위하여 함수를 정의해주었다.
- count_facility_df() : 반경 300m내에 존재하는 시설물 개수를 카운트하는 함수. 이 경우 중복된 좌표가 많지 않은 데이터프레임에 적합하여 다음과 같은 데이터를 할당하는데 사용하였다. (8개)
>어린이보호구역, 초등학교개수, 유치원/어린이집개수, 무인교통단속카메라개수, 과속방지턱개수, 버스정류장개수, 체육시설개수, 도로안전표지판개수
- count_facility_df_group() : 좌표가 중복된 행이 많은 경우 연산 과정을 줄이기 위해 위도와 경도로 그룹을 묶어 새로운 컬럼 count에 중복된 개수를 삽입하여준다. 다음의 데이터를 할당하는데 사용하였다. (4개)
>주정차단속건수, 학원교습소개수, 어린이보호구역CCTV개수, 유동인구
- count_geo_square() : GeoJson의 격자(5개의 coordinates)의 중심점을 추출하여 기존 4510개의 중심점과의 거리가 300m이내이면 해당 격자의 값(예를 들어 고령인구 데이터에서 val을 추출)을 추가해주는 함수이다. 다음의 데이터를 할당하는데 사용하였다.(4개)
>유소년인구수, 고령인구수, 생산가능인구수, 건물연면적

- count_geo_break() : Multiploygon 데이터 특성상 정확한 지점을 찍어주는것이 불가능하므로 coordinates 좌표중 한 개의 좌표(위도,경도)라도 반경 300m안에 있으면 cnt+1을 해주어 할당한다. 다음의 데이터를 할당하는데 사용하였다. (2개)
>횡단보도개수, 건물개수

- 개별 할당 코드 : 다음 데이터는 구조가 다른 GeoJson데이터와 비교하여 다르기 때문에 각 변수를 할당해주는 코드를 따로 작성하였다.(2개)
>신호등개수, 차량개수

- osan_accident 데이터의 위도 경도를 추출하여 기존 4510개(반경 300m) 의 격자 데이터에 사고횟수를 할당하여준다

- Polygon화
1. 기존에 100mx100m 의 직사각형 중심점을 기준으로 하여 300m 반경의 원을 형성
2. 오산시 도로에 있어 면적개념을 반영해주기 위하여 MultiLineString을 Polygon객체로 변환
3. 300m 반경의 원의 Polygon과 오산시 도로 Polygon의 intersection이 존재하면, 해당 지역에는 그 도로가 포함된다고 가정하여 해당 도로에 대한 교통량을 추가. 다음의 데이터를 할당하는데 사용하였다. (3개)
> 전체 추정교통량, 혼잡빈도강도, 혼잡시간강도





In [ ]:
def count_facility_df(df, attach_df, column_name): 
    arr=  []
    for i in range(len(df)):#각 좌표의 중심점과 시설의 좌표 사이의 거리를 계산하여 300m이내에 있으면 count+1
        if(i%1000==0):    #반복문의 진행상황을 알 수 있음.
            print(i,'/',4510)
        R = 6373.0
        cnt = 0
        lat1 = radians(df.iloc[i]['center_lat'])   
        lon1 = radians(df.iloc[i]['center_lon'])
        for j in range(len(attach_df)): 
            lat2 = radians(attach_df.iloc[j]['lat'])
            lon2 = radians(attach_df.iloc[j]['lon'])
            dlon = lon2 - lon1
            dlat = lat2 - lat1
            a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
            c = 2 * atan2(sqrt(a), sqrt(1 - a))
            distance = R * c
            if (distance <= 0.3):
                cnt = cnt+1
            else:
                continue
        arr.append(cnt)
    df[column_name]= arr
    print(column_name,'완료')
    display(df)
    
def count_facility_df_group(df, attach_df, column_name):   
    arr=  []
    for i in range(len(df)):#각 좌표의 중심점과 시설의 좌표 사이의 거리를 계산하여 300m이내에 있으면 count+df['column']
        if(i%1000==0):   
            print(i,'/',4510)
        R = 6373.0
        cnt = 0
        lat1 = radians(df.iloc[i]['center_lat'])   
        lon1 = radians(df.iloc[i]['center_lon'])
        for j in range(len(attach_df)):  
            lat2 = radians(attach_df.iloc[j]['lat'])
            lon2 = radians(attach_df.iloc[j]['lon'])
            dlon = lon2 - lon1
            dlat = lat2 - lat1
            a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
            c = 2 * atan2(sqrt(a), sqrt(1 - a))
            distance = R * c
            if (distance <= 0.3):
                cnt = cnt+attach_df.iloc[j]['count']
            else:
                continue
        arr.append(cnt)
    df[column_name]= arr
    print(column_name,'완료')
    display(df)
    
def count_geo_square(df, attach_df, column_name):   
    arr = []
    for i in range(len(df)):
        if(i%1000==0):    
            print(i,'/',4510)
        R = 6373.0
        cnt = 0
        lat1 = radians(df.iloc[i]['center_lat'])
        lon1 = radians(df.iloc[i]['center_lon'])
        for j in range(len(attach_df['features'])): #len(attach_df['features'])
            coord = attach_df['features'][j]['geometry']['coordinates'][0][0]
            lat2 = radians((coord[0][1] + coord[1][1])/2)    #격자의 중심점
            lon2 = radians((coord[1][0] + coord[2][0])/2)
            dlon = lon2 - lon1
            dlat = lat2 - lat1
            a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
            c = 2 * atan2(sqrt(a), sqrt(1 - a))
            distance = R * c
            if (distance <= 0.3):
                if attach_df['features'][j]['properties']['val']==None:
                    continue
                else:
                    cnt = cnt+ int(attach_df['features'][j]['properties']['val'])
            else:
                continue
        arr.append(cnt)
    df[column_name] = arr
    display(df)
    
def count_geo_break(df, attach_df, column_name):
    arr=  []
    for i in range(len(df)):
        if(i%1000==0):    #반복문의 진행상황을 알 수 있음.
            print(i,'/',4510)
        R = 6373.0
        cnt = 0
        lat1 = radians(df.iloc[i]['center_lat'])
        lon1 = radians(df.iloc[i]['center_lon'])
        for j in range(len(attach_df['features'])): #len(attach_df['features'])
            for k in range(len(attach_df['features'][j]['geometry']['coordinates'][0][0])-1):
                lat2 = radians(attach_df['features'][j]['geometry']['coordinates'][0][0][k][1])
                lon2 = radians(attach_df['features'][j]['geometry']['coordinates'][0][0][k][0])
                dlon = lon2 - lon1
                dlat = lat2 - lat1
                a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
                c = 2 * atan2(sqrt(a), sqrt(1 - a))
                distance = R * c
                if (distance <= 0.3):
                    cnt = cnt+1
                    break
                else:
                    continue
        arr.append(cnt)
    df[column_name] = arr
    display(df)

In [ ]:
#count_facility_df에 할당할 데이터
count_facility_df(df,df10,'초등학교개수')
count_facility_df(df,df13,'유치원/어린이집개수')
count_facility_df(df,df15,'무인교통단속카메라개수')
count_facility_df(df,df18,'과속방지턱개수')
count_facility_df(df,df22,'버스정류장개수')
count_facility_df(df,df29,'체육시설개수')
count_facility_df(df,df9,'어린이보호구역개수')
count_facility_df(df,df16,'도로안전표지판개수')
df

In [ ]:
#count_facility_df_group에 할당할 데이터 *시간이 매우 오래 걸림
count_facility_df_group(df, df1, '주정차단속건수')
count_facility_df_group(df, df9, '어린이보호구역CCTV개수')
count_facility_df_group(df, df8_2, '유동인구')
count_facility_df_group(df, df30, '학원교습소개수')


In [ ]:
#count_geo_square에 할당할 데이터 
count_geo_square(df, df4,'유소년인구수')
count_geo_square(df, df5, '고령인구수')
count_geo_square(df, df6, '생산가능인구수')
count_geo_square(df, df28, '건물연면적')


In [ ]:
#count_geo_break에 할당할 데이터
count_geo_break(df, df17, '횡단보도개수')
count_geo_break(df, df27, '건물개수')

In [ ]:
#개별 할당 코드 : 다음 데이터는 구조가 다른 GeoJson데이터와 비교하여 다르기 때문에 각 변수를 할당해주는 코드를 따로 작성하였다.
sinho_arr=  []
for i in range(len(df)):
    R = 6373.0
    cnt = 0
    lat1 = radians(df.iloc[i]['center_lat'])
    lon1 = radians(df.iloc[i]['center_lon'])
    for j in range(len(df19['features'])): 
        lat2 = radians(df19['features'][j]['geometry']['coordinates'][1])
        lon2 = radians(df19['features'][j]['geometry']['coordinates'][0])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        if (distance <= 0.3):
            cnt = cnt+1
        else:
            continue
    sinho_arr.append(cnt)
df['신호등개수'] = sinho_arr

car=  []
for i in range(len(df)):
    if(i%1000==0):    #반복문의 진행상황을 알 수 있음.
        print(i,'/',4510)
    R = 6373.0
    cnt = 0
    lat1 = radians(df.iloc[i]['center_lat'])
    lon1 = radians(df.iloc[i]['center_lon'])
    for j in range(100):#len(df3['features'])
        coord = df3['features'][j]['geometry']['coordinates'][0][0]
        lat2 = radians((coord[0][1] + coord[1][1])/2)
        lon2 = radians((coord[1][0] + coord[2][0])/2)
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        if (distance <= 0.3):
            if df3['features'][j]['properties']['car_cnt']==None:
                continue
            else:
                cnt = cnt+ int(df3['features'][j]['properties']['car_cnt'])
        else:
            continue
    car.append(cnt)
df['차량개수'] = car

In [ ]:
#TAAS 어린이 교통사고 데이터를 추출하여 기존 4510개의 격자에 사고횟수를 할당하여 준다.
osan_accident = pd.read_excel('osan_accident.xlsx')[['좌표']]
osan_accident
accident=  []
for i in range(len(df)):
    if(i%1000==0):    #반복문의 진행상황을 알 수 있음.
        print(i,'/',4510)
    R = 6373.0
    cnt = 0
    lat1 = radians(df.iloc[i]['center_lat'])
    lon1 = radians(df.iloc[i]['center_lon'])
    for j in range(len(osan_accident)):
        lat2 = radians(float(osan_accident['좌표'][j].split(',')[0]))
        lon2 = radians(float(osan_accident['좌표'][j].split(',')[1]))
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        if (distance <= 0.3):
            cnt = cnt+1
        else:
            continue
    accident.append(cnt)
df['사고횟수'] = accident

In [ ]:
#Polygon 객체 할당
# 300m 반경을 위한 함수 구현
#교통량 할당
proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')
def geodesic_point_buffer(lat, lon, km):
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return transform(project, buf).exterior.coords[:]

# 중심점 형성
df_square = pd.DataFrame(columns=['경도','위도'])
for i in range(len(df4['features'])):
    coord = df4['features'][i]['geometry']['coordinates'][0][0]
    center_lat = (coord[0][1] + coord[1][1])/2
    center_lon = (coord[1][0] + coord[2][0])/2
    df_square = df_square.append({'경도':center_lon, '위도':center_lat}, ignore_index=True) 

# 중심점을 Point 객체로 변환 후 중심점 데이터와 결합
geometry = [Point(xy) for xy in zip(df_square['경도'], df_square['위도'])]
df1 = gpd.GeoDataFrame(df_square, geometry=geometry)

# 좌표를 Polygon객체로 바꾼 후 저장
tmp_list = []
for i in range(len(df4['features'])):
    tmp = geodesic_point_buffer(df1['위도'][i], df1['경도'][i], 0.3)
    tmp_df = pd.DataFrame(tmp)
    geometry = [Point(xy) for xy in zip(tmp_df[0], tmp_df[1])]
    tmp_Polygon = Polygon(geometry)
    tmp_list.append(tmp_Polygon)    
df_300m = gpd.GeoDataFrame(df_square, geometry=gpd.GeoSeries(tmp_list))
df_300m

- 오산시 상세교통망 Multistring을 Polygon으로 변경

In [ ]:
# geopandas를 활용하여 geojson --> csv
df23_1 = gpd.read_file('23.오산시_상세도로망_LV6.geojson')

# multilinestring to buffer
for i in range(len(df23_1)):
    df23_1.loc[i,'geometry'] = df23_1.loc[i,'geometry'].buffer(0.00005)

df23_1[['link_id','geometry']]

- 전체 추정교통량 할당

In [ ]:
# 14~20시만 필터링
df24_1 = df24[df24['시간적범위'].isin(['15','16','17','18','19','20'])]

# LinkID로 groupby 후 Link_ID 뒤 두자리 삭제
df24_2 = df24_1.groupby('상세도로망_LinkID').mean()[['전체 추정교통량']]
df24_2 = df24_2.reset_index()

for i in range(len(df24_2)):
    df24_2.loc[i,'상세도로망_LinkID'] = str(df24_2.iloc[i,0])[:-2]

# 전체추정교통량과 merge
df24_3 = df24_2.groupby('상세도로망_LinkID').mean()[['전체 추정교통량']]
df24_3 = df24_3.reset_index()
df24_3 = df24_3.rename(columns={'상세도로망_LinkID':'link_id'})

df_traffic = pd.merge(df23_1[['link_id','geometry']], df24_3, on='link_id', how='outer')
df_traffic = df_traffic.dropna()

# 전체 추정교통량을 0으로 초기화
df_300m['전체 추정교통량'] = 0

# 격자할당
df_traffic = df_traffic.reset_index(drop=True)
for k in range(len(df_traffic)):
    buffer = df_traffic['geometry'][k].buffer(0.0001)
        
    for i in list(df_300m[df_300m['geometry'].intersection(buffer).is_empty == False]['전체 추정교통량'].index):
        df_300m.loc[i,'전체 추정교통량'] += df_traffic.loc[k,'전체 추정교통량']

- 혼잡빈도강도 할당

In [ ]:
# 혼잡빈도강도 초기화
df_300m['혼잡빈도강도'] = 0

# LinkID로 groupby 후 Link_ID 뒤 두자리 삭제
df25_1 = df25[['상세도로망_LinkID','혼잡빈도강도']]
df25_2 = df25_1.groupby('상세도로망_LinkID').mean()[['혼잡빈도강도']]
df25_2 = df25_2.reset_index()

for i in range(len(df25_2)):
    df25_2.loc[i,'상세도로망_LinkID'] = str(df25_2.iloc[i,0])[:-2]
    
# 상세교통망과 혼잡빈도강도를 merge
df25_3 = df25_2.groupby('상세도로망_LinkID').mean()[['혼잡빈도강도']]
df25_3 = df25_3.reset_index()
df25_3 = df25_3.rename(columns={'상세도로망_LinkID':'link_id'})

df_traffic = pd.merge(df23_1[['link_id','geometry']], df25_3, on='link_id', how='outer')
df_traffic = df_traffic.dropna()
df_traffic = df_traffic.reset_index(drop=True)

# 격자할당
for k in range(len(df_traffic)):
    buffer = df_traffic['geometry'][k].buffer(0.0001)
        
    for i in list(df_300m[df_300m['geometry'].intersection(buffer).is_empty == False]['혼잡빈도강도'].index):
        df_300m.loc[i,'혼잡빈도강도'] += df_traffic.loc[k,'혼잡빈도강도']

- 혼잡시간강도 할당

In [ ]:
# 혼잡시간강도 초기화
df_300m['혼잡시간강도'] = 0

# LinkID로 groupby 후 Link_ID 뒤 두자리 삭제
df26_1 = df26[['상세도로망_LinkID','혼잡시간강도']]
df26_2 = df26_1.groupby('상세도로망_LinkID').mean()[['혼잡시간강도']]
df26_2 = df26_2.reset_index()

for i in range(len(df26_2)):
    df26_2.loc[i,'상세도로망_LinkID'] = str(df26_2.iloc[i,0])[:-2]
    
# 상세교통망과 혼잡빈도강도를 merge
df26_3 = df26_2.groupby('상세도로망_LinkID').mean()[['혼잡시간강도']]
df26_3 = df26_3.reset_index()
df26_3 = df26_3.rename(columns={'상세도로망_LinkID':'link_id'})

df_traffic = pd.merge(df23_1[['link_id','geometry']], df26_3, on='link_id', how='outer')
df_traffic = df_traffic.dropna()
df_traffic = df_traffic.reset_index(drop=True)

# 격자할당
for k in range(len(df_traffic)):
    buffer = df_traffic['geometry'][k].buffer(0.0001)
        
    for i in list(df_300m[df_300m['geometry'].intersection(buffer).is_empty == False]['혼잡시간강도'].index):
        df_300m.loc[i,'혼잡시간강도'] += df_traffic.loc[k,'혼잡시간강도']
        
df_300m

- 최종 데이터프레임에 추정교통량과 혼잡빈도강도, 혼잡시간강도 컬럼을 할당하여준다.

In [ ]:
df['전체 추정교통량'] = df_300m['전체 추정교통량'].values
df['혼잡빈도강도'] = df_300m['혼잡빈도강도'].values
df['혼잡시간강도'] = df_300m['혼잡시간강도'].values
df.to_csv('2_DataFrame_final.csv', encoding='cp949', index=False)
df

In [ ]:
df = pd.read_csv('2_DataFrame_final.csv', encoding='cp949')

## 3) 사고횟수를 종속변수로 설정, 나머지 변수들을 독립변수로 하여 정규화를 진행 후 다중회귀 분석을 진행하였고, p-value가 0.1이하인 변수와 다중공선성 문제(VIF 기준 7이상)을 발생시키는 변수를 제거하여 새로운 데이터프레임을 구축하였다.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm
from sklearn import preprocessing
# X ,y분류
df_final = df.iloc[:,2:]
X = df_final.drop('사고횟수',axis=1)
y = df_final['사고횟수']

# 정규화
min_max_scaler = preprocessing.MinMaxScaler()
column_name = X.columns
X = min_max_scaler.fit_transform(X)
X = pd.DataFrame(X, columns = column_name)
display(X.head())
X = add_constant(X)


- 다중공선성을 확인하여 변수의 독립성을 해치는 변수 제거

In [ ]:
# 다중공선성 확인 (기준: 7)
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
display(vif)

#VIF 값이 가장 높은 유소년인구수 제거
X = X.drop('유소년인구수',axis=1)
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
display(vif)



In [ ]:
#VIF 값이 가장 높은 혼잡시간강도 제거
X = X.drop('혼잡시간강도',axis=1)
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
display(vif)

In [ ]:
#VIF 값이 가장 높은 생산가능인구수 제거
X = X.drop('생산가능인구수',axis=1)
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
display(vif)

In [ ]:
#VIF 값이 가장 높은 어린이보호구역CCTV개수 제거
X = X.drop('어린이보호구역CCTV개수',axis=1)
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
display(vif)

In [ ]:
#VIF 값이 가장 높은 유치원/어린이집개수 제거
X = X.drop('유치원/어린이집개수',axis=1)
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
display(vif)

In [ ]:
#다중회귀분석
model = sm.OLS(y, X)
fit_model = model.fit()
fit_model.summary(alpha=0.1)

In [ ]:
# p-value가 0.1이상인 ['무인교통단속카메라개수','주정차단속건수', '차량개수'] 제거 
remove_col = ['무인교통단속카메라개수', '주정차단속건수', '차량개수']
X = X.drop(remove_col,axis=1)

# 제거 후 P-Value 다시 check
model = sm.OLS(y, X)
fit_model = model.fit()
fit_model.summary(alpha=0.1)

In [ ]:
# 다중공선성 마지막 확인
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
display(vif)

## 4) 각 변수의 회귀계수를 정규화(Coefficient를 절댓값화 후 이들의 합으로 나눠줌) 시켜주어 가중치로 결정하였다.

In [ ]:
# coef datafrmae 형성
coef = fit_model.params
df_coef = pd.DataFrame(coef, columns=['coef'])
df_coef = df_coef.drop('const')

# 절대값 비중 계산
df_coef['abs_coef'] = abs(df_coef['coef'])

# 절대값 비중의 weight비중 도출
df_coef['weight'] = df_coef['abs_coef'] / df_coef.sum()[1]

# 인덱스 재정렬
df_coef = df_coef.reset_index()
df_coef = df_coef.rename(columns={'index':'변수'})

df_coef

# 91개의 후보군(어린이보호구역)에 대해 TOPSIS기법을 적용한다.
## 5) 앞선 과정과 동일하게 91개의 후보군(기존 어린이보호구역)에 대하여 같은 절차를 반복한다.
- 과정은 동일하지만, 유의미한 변수만 추출한다. 유의미한 변수란 앞선 과정에서 VIF와 P-value 체크를 통해 제거되지 않은 변수를 의미한다.

In [ ]:
df_91 = df9[['lon','lat']].rename(columns={'lon':'center_lon','lat':'center_lat'})
df_91

In [ ]:
count_facility_df(df_91,df10,'초등학교개수')
count_facility_df(df_91,df18,'과속방지턱개수')
count_facility_df(df_91,df22,'버스정류장개수')
count_facility_df(df_91,df29,'체육시설개수')
count_facility_df(df_91,df9,'어린이보호구역개수')
count_facility_df(df_91,df16,'도로안전표지판개수')

count_facility_df_group(df_91, df8_2, '유동인구')
count_facility_df_group(df_91, df30, '학원교습소개수')

count_geo_square(df_91, df5, '고령인구수')
count_geo_square(df_91, df28, '건물연면적')

count_geo_break(df_91, df17, '횡단보도개수')
count_geo_break(df_91, df27, '건물개수')

# ***91개이기 때문에 반복문 진행상황이 체크되지 않고 실행됩니다. 에러가 아님을 알립니다.

In [ ]:
sinho_arr=  []
for i in range(len(df_91)):
    R = 6373.0
    cnt = 0
    lat1 = radians(df_91.iloc[i]['center_lat'])
    lon1 = radians(df_91.iloc[i]['center_lon'])
    for j in range(len(df19['features'])): 
        lat2 = radians(df19['features'][j]['geometry']['coordinates'][1])
        lon2 = radians(df19['features'][j]['geometry']['coordinates'][0])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        if (distance <= 0.3):
            cnt = cnt+1
        else:
            continue
    sinho_arr.append(cnt)
df_91['신호등개수'] = sinho_arr
df_91

In [ ]:
df23_1 = gpd.read_file('23.오산시_상세도로망_LV6.geojson')
for i in range(len(df23_1)):
    df23_1.loc[i,'geometry'] = df23_1.loc[i,'geometry'].buffer(0.00005)
df23_1[['link_id','geometry']]

proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')
def geodesic_point_buffer(lat, lon, km):
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return transform(project, buf).exterior.coords[:]

# 중심점을 Point 객체로 변환 후 중심점 데이터와 결합
geometry = [Point(xy) for xy in zip(df_91['center_lon'], df_91['center_lat'])]
df1 = gpd.GeoDataFrame(df_91, geometry=geometry)

# 좌표를 Polygon객체로 바꾼 후 저장
tmp_list = []
for i in range(len(df_91)):
    tmp = geodesic_point_buffer(df1['center_lat'][i], df1['center_lon'][i], 0.3)
    tmp_df = pd.DataFrame(tmp)
    geometry = [Point(xy) for xy in zip(tmp_df[0], tmp_df[1])]
    tmp_Polygon = Polygon(geometry)
    tmp_list.append(tmp_Polygon)    
df_300m_91 = gpd.GeoDataFrame(df_91, geometry=gpd.GeoSeries(tmp_list))

# 14~20시만 필터링
df24_1 = df24[df24['시간적범위'].isin(['15','16','17','18','19','20'])]

# LinkID로 groupby 후 Link_ID 뒤 두자리 삭제
df24_2 = df24_1.groupby('상세도로망_LinkID').mean()[['전체 추정교통량']]
df24_2 = df24_2.reset_index()

for i in range(len(df24_2)):
    df24_2.loc[i,'상세도로망_LinkID'] = str(df24_2.iloc[i,0])[:-2]

# 전체추정교통량과 merge
df24_3 = df24_2.groupby('상세도로망_LinkID').mean()[['전체 추정교통량']]
df24_3 = df24_3.reset_index()
df24_3 = df24_3.rename(columns={'상세도로망_LinkID':'link_id'})

df_traffic = pd.merge(df23_1[['link_id','geometry']], df24_3, on='link_id', how='outer')
df_traffic = df_traffic.dropna()

# 전체 추정교통량을 0으로 초기화
df_300m_91['전체 추정교통량'] = 0

# 격자할당
df_traffic = df_traffic.reset_index(drop=True)
for k in range(len(df_traffic)):
    buffer = df_traffic['geometry'][k].buffer(0.0001)
        
    for i in list(df_300m_91[df_300m_91['geometry'].intersection(buffer).is_empty == False]['전체 추정교통량'].index):
        df_300m_91.loc[i,'전체 추정교통량'] += df_traffic.loc[k,'전체 추정교통량']
        
# 혼잡빈도강도 초기화
df_300m_91['혼잡빈도강도'] = 0

# LinkID로 groupby 후 Link_ID 뒤 두자리 삭제
df25_1 = df25[['상세도로망_LinkID','혼잡빈도강도']]
df25_2 = df25_1.groupby('상세도로망_LinkID').mean()[['혼잡빈도강도']]
df25_2 = df25_2.reset_index()

for i in range(len(df25_2)):
    df25_2.loc[i,'상세도로망_LinkID'] = str(df25_2.iloc[i,0])[:-2]
    
# 상세교통망과 혼잡빈도강도를 merge
df25_3 = df25_2.groupby('상세도로망_LinkID').mean()[['혼잡빈도강도']]
df25_3 = df25_3.reset_index()
df25_3 = df25_3.rename(columns={'상세도로망_LinkID':'link_id'})

df_traffic = pd.merge(df23_1[['link_id','geometry']], df25_3, on='link_id', how='outer')
df_traffic = df_traffic.dropna()
df_traffic = df_traffic.reset_index(drop=True)

# 격자할당
for k in range(len(df_traffic)):
    buffer = df_traffic['geometry'][k].buffer(0.0001)
        
    for i in list(df_300m_91[df_300m_91['geometry'].intersection(buffer).is_empty == False]['혼잡빈도강도'].index):
        df_300m_91.loc[i,'혼잡빈도강도'] += df_traffic.loc[k,'혼잡빈도강도']

In [ ]:
df_300m_91 = df_300m_91.drop(['geometry'], axis=1) #geometry 컬럼은 불필요하여 제거
df_300m_91

## 6) 각 컬럼들을 정규화 할 때, 다중회귀계수가 양이면 이익요소, 음이면 비용요소로 구분하였고 비례점수법과 벡터정규화법을 통해 정규화를 진행해주었다.

In [ ]:
# 비용요소와 이익요소는 방향성이 다르기에 분류
coef_minus_list = list(df_coef[df_coef['coef'] < 0].변수)
coef_minus = df_300m_91.iloc[:,2:][coef_minus_list]
coef_plus = df_300m_91.iloc[:,2:].drop(coef_minus_list,axis=1)
coef_minus.shape, coef_plus.shape

In [ ]:
# Coefficient가 음수면 클수록 안좋으며, 양수면 클수록 좋음.
## 이를 반영하기 위해 비례점수법으로 방향성을 일치화
### 스케일링을 실시하는 함수생성

# 비용요소 비례점수법 함수
def worst_scaling(colname):
    worst = max(coef_minus[colname])
    best = min(coef_minus[colname])
    result = (coef_minus[colname] - worst) / (best-worst)
    return result
    
# 이익요소 비례점수법 함수
def best_scaling(colname):
    best = max(coef_plus[colname])
    worst = min(coef_plus[colname])
    result = (coef_plus[colname] - worst) / (best-worst) 
    return result

In [ ]:
# Coefficient가 음수인 것들을 비례점수법
coef_minus_scaled = pd.concat([worst_scaling(coef_minus_list[0]),
                               worst_scaling(coef_minus_list[1]),
                               worst_scaling(coef_minus_list[2]),
                               worst_scaling(coef_minus_list[3])], axis=1)


# coef가 양수인 것들을 비례점수법
coef_plus_list = list(coef_plus.columns)
coef_plus_scaled = pd.concat([best_scaling(coef_plus_list[0]),
                               best_scaling(coef_plus_list[1]),
                               best_scaling(coef_plus_list[2]),
                               best_scaling(coef_plus_list[3]),
                               best_scaling(coef_plus_list[4]),
                               best_scaling(coef_plus_list[5]),
                               best_scaling(coef_plus_list[6]),
                               best_scaling(coef_plus_list[7]),
                               best_scaling(coef_plus_list[8]),
                               best_scaling(coef_plus_list[9]),
                               best_scaling(coef_plus_list[10])], axis=1)

# 스케일링 한 것들 concat
df_coef2 = pd.concat([coef_minus_scaled, coef_plus_scaled],axis=1)
df_coef2

In [ ]:
# 벡터정규화를 위한 함수구성
def vector_scaling(colname):
    under = math.sqrt((df_coef2[colname].head() ** 2).sum())
    result = df_coef2[colname] / under
    return result


# 벡터 정규화
colnames = list(df_coef2.columns)
df_vector_scaled = pd.concat([vector_scaling(colnames[0]),
                             vector_scaling(colnames[1]),
                             vector_scaling(colnames[2]),
                             vector_scaling(colnames[3]),
                             vector_scaling(colnames[4]),
                             vector_scaling(colnames[5]),
                             vector_scaling(colnames[6]),
                             vector_scaling(colnames[7]),
                             vector_scaling(colnames[8]),
                             vector_scaling(colnames[9]),
                             vector_scaling(colnames[10]),
                             vector_scaling(colnames[11]),
                             vector_scaling(colnames[12]),
                             vector_scaling(colnames[13]),
                             vector_scaling(colnames[14])], axis=1)

df_vector_scaled

## 7) 가중치와 정규화된 값들을 곱해 새로운 데이터프레임을 구축한다.

In [ ]:
# coef * 정규화 값을 곱해주기
df_coef_weight = df_coef.set_index('변수')
for name in (list(df_vector_scaled.columns)):
    df_vector_scaled.loc[:,name] = df_vector_scaled.loc[:,name] * df_coef_weight.loc[name,'weight']

df_vector_scaled

In [ ]:
# 각 변수 당 최선의 해(IS) 및 최악의 해(NIS)도출  
IS = df_vector_scaled.max()
IS = pd.DataFrame(data=IS).T
NIS = df_vector_scaled.min()
NIS = pd.DataFrame(data=NIS).T

IS_NIS_df = pd.concat([IS,NIS])
IS_NIS_df['index']=['IS','NIS']
IS_NIS_df = IS_NIS_df.set_index('index')
IS_NIS_df

## 8) 각 컬럼별 최대값의 집합을 "최선의 해(IS)", 최소값의 집합을 "최악의 해(NIS)"로 명시하고, 괴리도와 상대적 근접도를 통해 각 Instance의 IS와 NIS까지 거리를 산출한다.

In [ ]:
# 괴리도(상대적 근접도) 계산 (91개와 최악 및 최선의 해 거리계산) --> d_i+, d_i- 도출
import numpy as np
# 유클리디안 거리를 도출하기 위한 함수 형성
def euclidean_distance(x, y):   
    return np.sqrt(np.sum((x - y) ** 2))

# IS와 NIS까지 거리계산
IS_value = np.array(IS_NIS_df.loc['IS',:])
NIS_value = np.array(IS_NIS_df.loc['NIS',:])

dist_list = []
for i in range(len(df_vector_scaled)):
    value = np.array(df_vector_scaled.iloc[i,:])
    IS_value = np.array(IS_NIS_df.loc['IS',:])
    NIS_value = np.array(IS_NIS_df.loc['NIS',:])
    IS_dist = euclidean_distance(value, IS_value)
    NIS_dist = euclidean_distance(value, NIS_value)
    dist_list.append((IS_dist,NIS_dist))
    
df_dist = pd.DataFrame(dist_list, columns=['IS_dist','NIS_dist'])
df_dist

In [ ]:
# 상대적 근접도 계산

# 초기화
df_dist['c_plus'] = np.nan
df_dist['c_minus'] = np.nan

# c+, c- 도출
df_dist['c_plus'] = df_dist['IS_dist'] / (df_dist['IS_dist'] + df_dist['NIS_dist'])
df_dist['c_minus'] = df_dist['NIS_dist'] / (df_dist['IS_dist'] + df_dist['NIS_dist'])
df_dist

In [ ]:
final = df_dist.sort_values(by='c_minus', ascending=False).head(20)
final

## 9) IS와 가까울수록 최선의 해와 가까우며, 이는 곧 시설물이 우선적으로 설치가 필요하다고 판단하여 IS의 상위 20개 도출

In [ ]:
# 우선설치구역 20개의 시설물정보 불러오기

final_20 = df_300m_91.loc[final.index,:]
final_20 = pd.concat([df_91.loc[final_20.index,['center_lon','center_lat']], final_20], axis=1).reset_index(drop=True)
final_20_real = final_20.iloc[:,2:]
final_20_real

In [ ]:
final_20_real.to_csv('df_최종본.csv', index=False, encoding='cp949')

# 교통안전시설물 우선설치구역의 설치시설 선정
- 후보: 주의표지, 규제표지, 지시표지, 미끄럼포장도로, 가드레일, 과속방지턱, 단속카메라
- 교통안전설치 메뉴얼을 기반으로 하여 적합한 특성을 대입하여 도출
- 100mx100m 격자를 활용하여 해당 격자안에 특정 유형의 사고가 집중된 곳에 집중

## TAAS데이터 불러오기 및 데이터 좌표할당, Polygon화

In [ ]:
# TAAS 데이터를 불러온 후 좌표 할당
df = pd.read_excel('오산시 어린이 교통사고 내역, 2010~2019.xlsx')

df['lat'] = 0
df['lon'] = 0
for i in range(len(df)):
    df.loc[i,'lat'] = float(df.loc[i,'좌표'].split(',')[0])
    df.loc[i,'lon'] = float(df.loc[i,'좌표'].split(',')[1].strip())

df.head(3)

In [ ]:
# 격자형 데이터의 중심점 도출 및 geometry변수 저장

df2_a = gpd.read_file('2.오산시_어린이교통사고_격자.geojson')
df2_a = df2_a.drop('accident_cnt',axis=1)

df2_b = pd.DataFrame(columns=['격자명','사고횟수'])
for i in range(len(df2['features'])):
    a = df2['features'][i]['properties']['gid']
    b = df2['features'][i]['properties']['accident_cnt']
    c = np.mean(np.array(df2['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[0]
    d = np.mean(np.array(df2['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[1]
    new_data = {'격자명':a, '사고횟수':b,'lon': c,'lat': d}
    df2_b = df2_b.append(new_data, ignore_index=True)

df2_a = pd.concat([df2_b[['격자명','lon','lat']],df2_a['geometry']],axis=1)
df2_a

In [ ]:
# 300m 반경을 위한 함수 구현
proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')
def geodesic_point_buffer(lat, lon, km):
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return transform(project, buf).exterior.coords[:]

# final_20으로 이름 수정
final_20 = final_20_real.copy()

# 중심점을 Point 객체로 변환 후 중심점 데이터와 결합
geometry = [Point(xy) for xy in zip(final_20['center_lon'], final_20['center_lat'])]
final_20 = gpd.GeoDataFrame(final_20, geometry=geometry)

# 좌표를 Polygon객체로 바꾼 후 저장
tmp_list = []
for i in range(len(final_20)):
    tmp = geodesic_point_buffer(final_20['center_lat'][i], final_20['center_lon'][i], 0.3)
    tmp_df = pd.DataFrame(tmp)
    geometry = [Point(xy) for xy in zip(tmp_df[0], tmp_df[1])]
    tmp_Polygon = Polygon(geometry)
    tmp_list.append(tmp_Polygon)    
final_20 = gpd.GeoDataFrame(final_20, geometry=gpd.GeoSeries(tmp_list))

# 격자명을 우선설치순서대로 1~20 부여
for i in range(len(final_20)):
    final_20.loc[i,'격자명'] = (i+1)
    
final_20

## 시각화(EDA)

### 1. 오산시 교통량 시각화
- 어린이가 가장 많이 다니는 14~20시의 전체 추정교통량 파악

In [ ]:
# geopandas를 활용하여 상세도로망 데이터를 geojson --> csv
df23_1 = gpd.read_file('23.오산시_상세도로망_LV6.geojson')

# multilinestring --> buffer 변환
for i in range(len(df23_1)):
    df23_1.loc[i,'geometry'] = df23_1.loc[i,'geometry'].buffer(0.00005)

df23_1.head(3)

In [ ]:
# 시간적 범위는 어린이가 가장 많이 몰리는 시간대인 14~20시로 한정
df24_1 = df24[df24['시간적범위'].isin(['15','16','17','18','19','20'])] 

# 상세도로망별 평균 "전체 추정교통량"도출
df24_2 = df24_1.groupby('상세도로망_LinkID').mean()[['전체 추정교통량']]
df24_2 = df24_2.reset_index()

# df23: 상세도로망 데이터와 연관짓기 위해 뒤에 상세도로망_LinkID2자리 삭제
for i in range(len(df24_2)):
    df24_2.loc[i,'상세도로망_LinkID'] = str(df24_2.iloc[i,0])[:-2]

# 뒤에 2자리를 지운 후 다시 groupby 실시
df24_3 = df24_2.groupby('상세도로망_LinkID').mean()[['전체 추정교통량']]
df24_3 = df24_3.reset_index()

# df23과 df24의 link_id의 열이름 통일
df24_3 = df24_3.rename(columns={'상세도로망_LinkID':'link_id'})

# link_id를 활용해 두 Dataframe을 merge
df_traffic = pd.merge(df23_1[['link_id','geometry']], df24_3, on='link_id', how='outer')
df_traffic = df_traffic.dropna()

df_traffic

In [ ]:
# 전체 추정교통량을 4분위수 단위로 나누어 교통량을 시각화

p1 = df_traffic['전체 추정교통량'].describe()['min']
p2 = df_traffic['전체 추정교통량'].describe()['25%']
p3 = df_traffic['전체 추정교통량'].describe()['50%']
p4 = df_traffic['전체 추정교통량'].describe()['75%']
p5 = df_traffic['전체 추정교통량'].describe()['max']

df_traffic1 = df_traffic[(df_traffic['전체 추정교통량'] >= p1) & (df_traffic['전체 추정교통량'] < p2)].reset_index(drop=True)
df_traffic2 = df_traffic[(df_traffic['전체 추정교통량'] >= p2) & (df_traffic['전체 추정교통량'] < p3)].reset_index(drop=True)
df_traffic3 = df_traffic[(df_traffic['전체 추정교통량'] >= p3) & (df_traffic['전체 추정교통량'] < p4)].reset_index(drop=True)
df_traffic4 = df_traffic[(df_traffic['전체 추정교통량'] >= p4) & (df_traffic['전체 추정교통량'] < p5)].reset_index(drop=True)

In [ ]:
# 시각화
m = folium.Map(
    location=[37.1516633,127.0359803],
    zoom_start=12
)


# 오산시 표시
folium.GeoJson(
    df31,
    name='osan'
).add_to(m)


# 도로망 표시 (원활)
folium.GeoJson(
    df_traffic1,
    name='road',
    style_function=lambda feature: {
        'color': 'Green'
    }
).add_to(m)

# 도로망 표시 (보통)
folium.GeoJson(
    df_traffic2,
    name='road',
    style_function=lambda feature: {
        'color': 'Yellow'
    }
).add_to(m)

# 도로망 표시 (부분정체)
folium.GeoJson(
    df_traffic3,
    name='road',
    style_function=lambda feature: {
        'color': 'Orange'
    }
).add_to(m)

# 도로망 표시 (완전정체)
folium.GeoJson(
    df_traffic4,
    name='road',
    style_function=lambda feature: {
        'color': 'Red'
    }
).add_to(m)

    
m

### 2. 유동인구 시각화
- 어린이가 가장 많이 다니는 14~20시의 유동인구 파악

In [ ]:
# 어린이가 많이 다니는 14시 ~ 20시로 한정
df8_1 = df8.loc[:,['lon','lat','TMST_14','TMST_15','TMST_16','TMST_17','TMST_18','TMST_19','TMST_20']]
df8_1['mean'] = (df8_1['TMST_14'] + df8_1['TMST_15'] + df8_1['TMST_16'] + df8_1['TMST_17'] + df8_1['TMST_18'] + df8_1['TMST_19'] +df8_1['TMST_20'])/7
df8_1 = df8_1.loc[:,['lon','lat','mean']]
df8_1

In [ ]:
# 위도와 경도기준 groupby
df8_2 = df8_1.groupby(['lon','lat']).sum()
df8_2 = df8_2.reset_index()
df8_2 = df8_2.reindex(columns=['lat','lon','mean'])
df8_2

In [ ]:
# 중심점을 Point 객체로 변환 후 중심점 데이터와 결합
geometry = [Point(xy) for xy in zip(df8_2['lon'], df8_2['lat'])]
df8_3 = gpd.GeoDataFrame(df8_2, geometry=geometry)
df8_3

In [ ]:
# 좌표를 Polygon객체로 바꾼 후 저장
tmp_list = []
for i in range(len(df8_3)):
    tmp = geodesic_point_buffer(df8_3['lat'][i], df8_3['lon'][i], 0.025)
    tmp_df = pd.DataFrame(tmp)
    geometry = [Point(xy) for xy in zip(tmp_df[0], tmp_df[1])]
    tmp_Polygon = Polygon(geometry)
    tmp_list.append(tmp_Polygon)    

df8_4 = gpd.GeoDataFrame(df8_3, geometry=gpd.GeoSeries(tmp_list))
df8_4

In [ ]:
# 유동인구를 4분위 수로 나누어 시각화

m1 = df8_4['mean'].describe()['min']
m2 = df8_4['mean'].describe()['25%']
m3 = df8_4['mean'].describe()['50%']
m4 = df8_4['mean'].describe()['75%']
m5 = df8_4['mean'].describe()['max']

df8_4_1 = df8_4[(df8_4['mean'] >= m1) & (df8_4['mean'] < m2)].reset_index(drop=True)
df8_4_2 = df8_4[(df8_4['mean'] >= m2) & (df8_4['mean'] < m3)].reset_index(drop=True)
df8_4_3 = df8_4[(df8_4['mean'] >= m3) & (df8_4['mean'] < m4)].reset_index(drop=True)
df8_4_4 = df8_4[(df8_4['mean'] >= m4) & (df8_4['mean'] < m5)].reset_index(drop=True)

In [ ]:
# 시각화
m = folium.Map(
    location=[37.1516633,127.0359803],
    zoom_start=12
)


# 오산시 표시
folium.GeoJson(
    df31,
    name='osan'
).add_to(m)


# 유동인구 표시1
for i in range(len(df8_4_1)):
    folium.GeoJson(
        df8_4_1.loc[i,'geometry'],
        name='people',
        style_function=lambda feature: {
        'color': 'Green'}
    ).add_to(m)
    
# 유동인구 표시2
for i in range(len(df8_4_2)):
    folium.GeoJson(
        df8_4_2.loc[i,'geometry'],
        name='people',
        style_function=lambda feature: {
        'color': 'Yellow'}
    ).add_to(m)

    
# 유동인구 표시3
for i in range(len(df8_4_3)):
    folium.GeoJson(
        df8_4_3.loc[i,'geometry'],
        name='people',
        style_function=lambda feature: {
        'color': 'Orange'}
    ).add_to(m)

    
# 유동인구 표시4
for i in range(len(df8_4_4)):
    folium.GeoJson(
        df8_4_4.loc[i,'geometry'],
        name='osan',
        style_function=lambda feature: {
        'color': 'Red'}
    ).add_to(m)


m.save('example.html')
m

### 3. 총인구수 시각화
- 격자의 인구수가 0인 부분을 제외하고 4분위수를 구하여 시각화

In [ ]:
# Nan 부분은 0으로 대체
df4_1 = gpd.read_file('4.오산시_연령별_거주인구격자(총인구).geojson')
df4_1 = df4_1.fillna(0)
df4_1

In [ ]:
# 총인구수가 0보다 큰 격자만 가져와 4분위수 도출
df4_1 = df4_1[df4_1['val'] > 0].reset_index(drop=True)

k1 = df4_1['val'].describe()['min']
k2 = df4_1['val'].describe()['25%']
k3 = df4_1['val'].describe()['50%']
k4 = df4_1['val'].describe()['75%']
k5 = df4_1['val'].describe()['max']

df4_1_1 = df4_1[(df4_1['val'] >= k1) & (df4_1['val'] < k2)].reset_index(drop=True)
df4_1_2 = df4_1[(df4_1['val'] >= k2) & (df4_1['val'] < k3)].reset_index(drop=True)
df4_1_3 = df4_1[(df4_1['val'] >= k3) & (df4_1['val'] < k4)].reset_index(drop=True)
df4_1_4 = df4_1[(df4_1['val'] >= k4) & (df4_1['val'] < k5)].reset_index(drop=True)

In [ ]:
# 시각화
m = folium.Map(
    location=[37.1516633,127.0359803],
    zoom_start=12
)

# 오산시 표시
folium.GeoJson(
    df31,
    name='osan'
).add_to(m)

# 총인구수1
for i in range(len(df4_1_1)):
    folium.GeoJson(
        df4_1_1.loc[i,'geometry'],
        name='road',
        style_function=lambda feature: {
            'color': 'Green'
        }
    ).add_to(m)

# 총인구수2
for i in range(len(df4_1_2)):
    folium.GeoJson(
        df4_1_2.loc[i,'geometry'],
        name='road',
        style_function=lambda feature: {
            'color': 'Yellow'
        }
    ).add_to(m)

# 총인구수3
for i in range(len(df4_1_3)):
    folium.GeoJson(
        df4_1_3.loc[i,'geometry'],
        name='road',
        style_function=lambda feature: {
            'color': 'Orange'
        }
    ).add_to(m)

# 총인구수4
for i in range(len(df4_1_4)):
    folium.GeoJson(
        df4_1_4.loc[i,'geometry'],
        name='road',
        style_function=lambda feature: {
            'color': 'Red'
        }
    ).add_to(m)

m.save('example.html')
m

### 4. 유소년 인구수 시각화

In [ ]:
# 유소년인구수 데이터를 불러온 후 Nan값은 0으로 대체
df5_1 = gpd.read_file('5.오산시_연령별_거주인구격자(유소년).geojson')
df5_1 = df5_1.fillna(0)
df5_1

In [ ]:
# 인구수가 0보다 큰 값만 가져와 4분위수 도출
df5_1 = df5_1[df5_1['val'] > 0].reset_index(drop=True)

y1 = df5_1['val'].describe()['min']
y2 = df5_1['val'].describe()['25%']
y3 = df5_1['val'].describe()['50%']
y4 = df5_1['val'].describe()['75%']
y5 = df5_1['val'].describe()['max']

df5_1_1 = df5_1[(df5_1['val'] >= y1) & (df5_1['val'] < y2)].reset_index(drop=True)
df5_1_2 = df5_1[(df5_1['val'] >= y2) & (df5_1['val'] < y3)].reset_index(drop=True)
df5_1_3 = df5_1[(df5_1['val'] >= y3) & (df5_1['val'] < y4)].reset_index(drop=True)
df5_1_4 = df5_1[(df5_1['val'] >= y4) & (df5_1['val'] < y5)].reset_index(drop=True)

In [ ]:
# 시각화
m = folium.Map(
    location=[37.1516633,127.0359803],
    zoom_start=12
)


# 오산시 표시
folium.GeoJson(
    df31,
    name='osan'
).add_to(m)

# 유소년인구수1
for i in range(len(df5_1_1)):
    folium.GeoJson(
        df5_1_1.loc[i,'geometry'],
        name='children',
        style_function=lambda feature: {
            'color': 'Green'
        }
    ).add_to(m)

# 유소년인구수2
for i in range(len(df5_1_2)):
    folium.GeoJson(
        df5_1_2.loc[i,'geometry'],
        name='children',
        style_function=lambda feature: {
            'color': 'Yellow'
        }
    ).add_to(m)

# 유소년인구수3
for i in range(len(df5_1_3)):
    folium.GeoJson(
        df5_1_3.loc[i,'geometry'],
        name='children',
        style_function=lambda feature: {
            'color': 'Orange'
        }
    ).add_to(m)

# 유소년인구수4
for i in range(len(df5_1_4)):
    folium.GeoJson(
        df5_1_4.loc[i,'geometry'],
        name='children',
        style_function=lambda feature: {
            'color': 'Red'
        }
    ).add_to(m)

m.save('example.html')
m

### 5. 초등학교 시각화
- 초등학교, 중학교, 고등학교 중 초등학교의 위치만 시각화

In [ ]:
# 학교정보에서 초등학교만 필터링
df10_a = df10[df10['학교구분'] == '초등학교'].reset_index(drop=True) 
df10_a

In [ ]:
# 시각화
m = folium.Map(
    location=[37.1516633,127.0359803],
    zoom_start=12
)

# 오산시 표시
folium.GeoJson(
    df31,
    name='osan'
).add_to(m)

# 초등학교 표시
for i in range(len(df10_a)):
    folium.CircleMarker([df10_a.loc[i,'학교위치_위도'], df10_a.loc[i,'학교위치_경도']], radius=6, color='Red').add_to(m)

m.save('example.html')
m

### 6. 어린이집/유치원 시각화

In [ ]:
# 시각화
import folium

m = folium.Map(
    location=[37.1516633,127.0359803],
    zoom_start=12
)

# 오산시 표시
folium.GeoJson(
    df31,
    name='child'
).add_to(m)

# 어린이집/유치원 표시
for i in range(len(df13)):
    folium.CircleMarker([df13.loc[i,'시설위치_위도'], df13.loc[i,'시설위치_경도']], radius=6, color='Red').add_to(m)

m.save('example.html')
m

### 7. 전체사고횟수 시각화

In [ ]:
# 시각화
import folium

m = folium.Map(
    location=[37.1516633,127.0359803],
    zoom_start=12
)


# 오산시 표시
folium.GeoJson(
    df31,
    name='osan'
).add_to(m)

# 사고횟수
for i in range(len(df)):
    folium.CircleMarker([df.loc[i,'lat'], df.loc[i,'lon']], radius=4, color='red').add_to(m)

# 20개 우선설치구역 표시
for i in range(len(final_20)):
    folium.vector_layers.Circle([final_20.loc[i,'center_lat'], final_20.loc[i,'center_lon']],radius=300,color='Black',
                               tooltip=final_20.loc[i,'격자명']).add_to(m)


    
m.save('example.html')
m

## 시설물 시각화

### 시각화1: 주의표지
- 법규위반 : 교차로운행방법위반, 신호위반, 
- 사고유형 : 횡단중, 공작물출동
- 해당 조건이 하나라도 포함되는 사고가 3건이상

In [ ]:
# 조건에 맞는 행 불러오기
cond1 = df['법규위반'].isin(['교차로운행방법위반','신호위반'])
cond2 = df['사고유형'].isin(['차대사람 - 횡단중','차량단독 - 공작물충돌'])

type_a = df[cond1 | cond2].reset_index(drop=True)
type_a.head(3)

In [ ]:
# 해당 유형에 속하는 사고를 격자화

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

type_a_count = pd.DataFrame({'격자명':list_tmp})
type_a_count['사고횟수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(type_a)):
        if (type_a.loc[j, 'lat'] >= min_lat) & (type_a.loc[j, 'lat'] <= max_lat) & (type_a.loc[j, 'lon'] >= min_lon) & (type_a.loc[j, 'lon'] <= max_lon):
            type_a_count.loc[i,'사고횟수'] += 1
            

type_a_count = pd.concat([type_a_count, df2_a[['geometry','lat','lon']]], axis=1)
type_a_count

In [ ]:
# 사고횟수가 3회이상인 격자만 필터링

type_a_count_final = type_a_count[type_a_count['사고횟수'] >= 3].reset_index(drop=True)       
type_a_count_final

In [ ]:
# 시각화
import folium

m = folium.Map(
    location=[37.1516633,127.0359803],
    zoom_start=12
)


# 오산시 표시
folium.GeoJson(
    df31,
    name='osan',
    style_function=lambda feature: {
        'color': 'grey'}
).add_to(m)


# 20개 우선설치구역 표시
for i in range(len(final_20)):
    folium.vector_layers.Circle([final_20.loc[i,'center_lat'], final_20.loc[i,'center_lon']],radius=300,color='Black',
                               tooltip=final_20.loc[i,'격자명']).add_to(m)

    
# 사고다발지역 표시
for i in range(len(type_a_count_final)):
    folium.GeoJson(
        type_a_count_final.loc[i,'geometry'],
        name='sago',
        tooltip=type_a_count_final.loc[i,'격자명'],
        style_function=lambda feature: {
        'color': 'Yellow'}
    ).add_to(m)

m.save('example.html')
m

In [ ]:
# 격자할당
R = 6373.0

# 초기값 생성
count_df1 = type_a_count_final[['격자명','lat','lon']]
count_df1['할당격자'] = 0
count_df1['할당_위도'] = 0
count_df1['할당_경도'] = 0
count_df1['시설물'] = '주의표지'

# 사고다발지역과 우선설치구역이 겹치는 것들 중 사고다발지역 격자의 중심점과 우선설치구역 20개 중심의 거리를 측정한 후 가장 가까운 곳에 시설물을 할당
for k in range(len(type_a_count_final)):
    buffer = type_a_count_final.loc[k,'geometry']
    lat1 = radians(type_a_count_final.loc[k,'lat'])
    lon1 = radians(type_a_count_final.loc[k,'lon'])
    dist_list = []
    index_list = []
    
    for i in list(final_20[final_20['geometry'].intersection(buffer).is_empty == False].index):
        lat2 = radians(final_20.loc[i,'center_lat'])
        lon2 = radians(final_20.loc[i,'center_lon'])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        dist_list.append(distance)
        index_list.append(i)
   
    if dist_list == []:
        continue
    else:
        close_distance = min(dist_list)
        index_tmp = dist_list.index(close_distance)
        optimal_index = index_list[index_tmp]
        count_df1.loc[k,'할당격자'] = final_20.loc[optimal_index, '격자명']
        count_df1.loc[k,'할당_위도'] = final_20.loc[optimal_index, 'center_lat']
        count_df1.loc[k,'할당_경도'] = final_20.loc[optimal_index, 'center_lon']

        
count_df1

### 시각화2: 규제표지
- 법규위반 : 직진우회전진행방해, 불법유턴, 안전거리미확보, 과속
- 사고유형 : 차도통행중
- 해당 조건이 하나라도 포함되는 사고가 1건이상

In [ ]:
# 조건에 맞는 행 불러오기
cond1 = df['법규위반'].isin(['직진우회전진행방해','불법유턴','안전거리미확보','과속'])
cond2 = df['사고유형'].isin(['차도통행중'])

type_b = df[cond1 | cond2].reset_index(drop=True)
type_b

In [ ]:
# 해당 유형에 속하는 사고를 격자화

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

type_b_count = pd.DataFrame({'격자명':list_tmp})
type_b_count['사고횟수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(type_b)):
        if (type_b.loc[j, 'lat'] >= min_lat) & (type_b.loc[j, 'lat'] <= max_lat) & (type_b.loc[j, 'lon'] >= min_lon) & (type_b.loc[j, 'lon'] <= max_lon):
            type_b_count.loc[i,'사고횟수'] += 1
            

type_b_count = pd.concat([type_b_count, df2_a[['geometry','lat','lon']]], axis=1)
type_b_count

In [ ]:
# 사고횟수가 1회이상인 격자만 필터링
type_b_count_final = type_b_count[type_b_count['사고횟수'] >= 1].reset_index(drop=True)       
type_b_count_final

In [ ]:
# 시각화
m = folium.Map(
    location=[37.1516633,127.0359803],
    zoom_start=12
)


# 오산시 표시
folium.GeoJson(
    df31,
    name='osan',
    style_function=lambda feature: {
        'color': 'grey'}
).add_to(m)


# 20개 우선설치구역 표시
for i in range(len(final_20)):
    folium.vector_layers.Circle([final_20.loc[i,'center_lat'], final_20.loc[i,'center_lon']],radius=300,color='Black',
                               tooltip=final_20.loc[i,'격자명']).add_to(m)

    
# 사고다발지역 표시
for i in range(len(type_b_count_final)):
    folium.GeoJson(
        type_b_count_final.loc[i,'geometry'],
        name='sago',
        tooltip=type_b_count_final.loc[i,'격자명'],
        style_function=lambda feature: {
        'color': 'Yellow'}
).add_to(m)

m.save('example.html')
m

In [ ]:
# 사고다발지역과 우선설치구역이 겹치는 것들 중 사고다발지역 격자의 중심점과 우선설치구역 20개 중심의 거리를 측정한 후 가장 가까운 곳에 시설물을 할당
R = 6373.0

#초기값 생성
count_df2 = type_b_count_final[['격자명','lat','lon']]
count_df2['할당격자'] = 0
count_df2['할당_위도'] = 0
count_df2['할당_경도'] = 0
count_df2['시설물'] = '규제표지'


for k in range(len(type_b_count_final)):
    buffer = type_b_count_final.loc[k,'geometry']
    lat1 = radians(type_b_count_final.loc[k,'lat'])
    lon1 = radians(type_b_count_final.loc[k,'lon'])
    dist_list = []
    index_list = []
    
    for i in list(final_20[final_20['geometry'].intersection(buffer).is_empty == False].index):
        lat2 = radians(final_20.loc[i,'center_lat'])
        lon2 = radians(final_20.loc[i,'center_lon'])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        dist_list.append(distance)
        index_list.append(i)
   
    if dist_list == []:
        continue
    else:
        close_distance = min(dist_list)
        index_tmp = dist_list.index(close_distance)
        optimal_index = index_list[index_tmp]
        count_df2.loc[k,'할당격자'] = final_20.loc[optimal_index, '격자명']
        count_df2.loc[k,'할당_위도'] = final_20.loc[optimal_index, 'center_lat']
        count_df2.loc[k,'할당_경도'] = final_20.loc[optimal_index, 'center_lon']

        
count_df2

### 시각화3: 지시표지
- 법규위반 :보행자보호의무위반, 교차로운행방법위반
- 해당 조건이 하나라도 포함되는 사고가 2건이상

In [ ]:
# 조건에 맞는 행 불러오기
cond1 = df['법규위반'].isin(['보행자보호의무위반','교차로운행방법위반'])

type_c = df[cond1].reset_index(drop=True)
type_c

In [ ]:
# 해당 유형에 속하는 사고를 격자화

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

type_c_count = pd.DataFrame({'격자명':list_tmp})
type_c_count['사고횟수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(type_c)):
        if (type_c.loc[j, 'lat'] >= min_lat) & (type_c.loc[j, 'lat'] <= max_lat) & (type_c.loc[j, 'lon'] >= min_lon) & (type_c.loc[j, 'lon'] <= max_lon):
            type_c_count.loc[i,'사고횟수'] += 1
            

type_c_count = pd.concat([type_c_count, df2_a[['geometry','lat','lon']]], axis=1)
type_c_count

In [ ]:
# 사고횟수가 2회이상인 격자만 필터링
type_c_count_final = type_c_count[type_c_count['사고횟수'] >= 2].reset_index(drop=True)       
type_c_count_final

In [ ]:
# 사고다발지역과 우선설치구역이 겹치는 것들 중 사고다발지역 격자의 중심점과 우선설치구역 20개 중심의 거리를 측정한 후 가장 가까운 곳에 시설물을 할당

# 시각화
m = folium.Map(
    location=[37.1516633,127.0359803],
    zoom_start=12
)


# 오산시 표시
folium.GeoJson(
    df31,
    name='osan',
    style_function=lambda feature: {
        'color': 'grey'}
).add_to(m)


# 20개 우선설치구역 표시
for i in range(len(final_20)):
    folium.vector_layers.Circle([final_20.loc[i,'center_lat'], final_20.loc[i,'center_lon']],radius=300,color='Black',
                               tooltip=final_20.loc[i,'격자명']).add_to(m)

    
# 사고다발지역 표시
for i in range(len(type_c_count_final)):
    folium.GeoJson(
        type_c_count_final.loc[i,'geometry'],
        name='sago',
        tooltip=type_c_count_final.loc[i,'격자명'],
        style_function=lambda feature: {
        'color': 'Yellow'}
).add_to(m)

m.save('example.html')
m

In [ ]:
# 격자할당
R = 6373.0

#초기값 생성
count_df3 = type_c_count_final[['격자명','lat','lon']]
count_df3['할당격자'] = 0
count_df3['할당_위도'] = 0
count_df3['할당_경도'] = 0
count_df3['시설물'] = '지시표지'


for k in range(len(type_c_count_final)):
    buffer = type_c_count_final.loc[k,'geometry']
    lat1 = radians(type_c_count_final.loc[k,'lat'])
    lon1 = radians(type_c_count_final.loc[k,'lon'])
    dist_list = []
    index_list = []
    
    for i in list(final_20[final_20['geometry'].intersection(buffer).is_empty == False].index):
        lat2 = radians(final_20.loc[i,'center_lat'])
        lon2 = radians(final_20.loc[i,'center_lon'])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        dist_list.append(distance)
        index_list.append(i)
   
    if dist_list == []:
        continue
    else:
        close_distance = min(dist_list)
        index_tmp = dist_list.index(close_distance)
        optimal_index = index_list[index_tmp]
        count_df3.loc[k,'할당격자'] = final_20.loc[optimal_index, '격자명']
        count_df3.loc[k,'할당_위도'] = final_20.loc[optimal_index, 'center_lat']
        count_df3.loc[k,'할당_경도'] = final_20.loc[optimal_index, 'center_lon']

        
count_df3

### 시각화4: 미끄럼포장도로

1. 사고횟수가 2회이상(사고다발지역)
2. 노면상태: '건조'제외

In [ ]:
# 노면상태가 건조인 것이외의 것만 가져오기

idx1 = df[df['노면상태'] == '포장 - 건조'].index
idx2 = df[df['노면상태'] == '비포장 - 건조'].index

a_1 = df.drop(idx1)
a_1 = a_1.drop(idx2)
a_1 = a_1.reset_index()
a_1

In [ ]:
# 사고횟수 할당

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

a_2 = pd.DataFrame({'격자명':list_tmp})
a_2['사고횟수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(a_1)):
        if (a_1.loc[j, 'lat'] >= min_lat) & (a_1.loc[j, 'lat'] <= max_lat) & (a_1.loc[j, 'lon'] >= min_lon) & (a_1.loc[j, 'lon'] <= max_lon):
            a_2.loc[i,'사고횟수'] = a_2.loc[i,'사고횟수']+1

a_2 = pd.concat([df2_a[['격자명','geometry','lat','lon']], a_2['사고횟수']], axis=1)
a_2

In [ ]:
# 사고횟수가 2회이상인 격자만 불러오기
a_3 = a_2[a_2['사고횟수'] >= 2].reset_index(drop=True)
a_3

In [ ]:
# 시각화
m = folium.Map(
    location=[37.1516633,127.0359803],
    zoom_start=12
)


# 오산시 표시
folium.GeoJson(
    df31,
    name='osan',
    style_function=lambda feature: {
        'color': 'grey'}
).add_to(m)


# 20개 우선설치구역 표시
for i in range(len(final_20)):
    folium.vector_layers.Circle([final_20.loc[i,'center_lat'], final_20.loc[i,'center_lon']],radius=300,color='Black',
                               tooltip=final_20.loc[i,'격자명']).add_to(m)

    
# 사고다발지역 표시
for i in range(len(a_3)):
    folium.GeoJson(
        a_3.loc[i,'geometry'],
        name='sago',
        style_function=lambda feature: {
        'color': 'Yellow'},
        tooltip=a_3.loc[i,'격자명']
).add_to(m)

m.save('example.html')
m

In [ ]:
# 격자할당
R = 6373.0

#초기값 생성
count_df4 = a_3[['격자명','lat','lon']]
count_df4['할당격자'] = 0
count_df4['할당_위도'] = 0
count_df4['할당_경도'] = 0
count_df4['시설물'] = '미끄럼포장도로'


for k in range(len(a_3)):
    buffer = a_3.loc[k,'geometry']
    lat1 = radians(a_3.loc[k,'lat'])
    lon1 = radians(a_3.loc[k,'lon'])
    dist_list = []
    index_list = []
    
    for i in list(final_20[final_20['geometry'].intersection(buffer).is_empty == False].index):
        lat2 = radians(final_20.loc[i,'center_lat'])
        lon2 = radians(final_20.loc[i,'center_lon'])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        dist_list.append(distance)
        index_list.append(i)
   
    if dist_list == []:
        continue
    else:
        close_distance = min(dist_list)
        index_tmp = dist_list.index(close_distance)
        optimal_index = index_list[index_tmp]
        count_df4.loc[k,'할당격자'] = final_20.loc[optimal_index, '격자명']
        count_df4.loc[k,'할당_위도'] = final_20.loc[optimal_index, 'center_lat']
        count_df4.loc[k,'할당_경도'] = final_20.loc[optimal_index, 'center_lon']

        
count_df4

## 시각화5: 가드레일
- 사고유형: '차대사람 - 길가장자리구역통행중' or '차대사람 - 보도통행중' or '차량단독 - 도로외이탈'
- 법규위반: '보행자보호의무위반'
- 해당 조건이 하나라도 포함되는 사고가 1건이상

In [ ]:
# 조건에 맞는 행 불러오기
cond1 = (df['사고유형'] == '차대사람 - 길가장자리구역통행중')
cond2 = (df['사고유형'] == '차대사람 - 보도통행중')
cond3 = (df['사고유형'] == '차량단독 - 도로외이탈')
cond4 = (df['법규위반'] == '보행자보호의무위반')
b_1 = df[cond1 | cond2 | cond3 | cond4].reset_index(drop=True)
b_1.shape

In [ ]:
# 해당 유형에 속하는 사고를 격자화
list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

b_2 = pd.DataFrame({'격자명':list_tmp})
b_2['사고횟수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(b_1)):
        if (b_1.loc[j, 'lat'] >= min_lat) & (b_1.loc[j, 'lat'] <= max_lat) & (b_1.loc[j, 'lon'] >= min_lon) & (b_1.loc[j, 'lon'] <= max_lon):
            b_2.loc[i,'사고횟수'] += 1
            

b_2 = pd.concat([b_2, df2_a[['geometry','lat','lon']]], axis=1)
b_2

In [ ]:
# 사고횟수가 1회이상인 격자만 불러오기
b_3 = b_2[b_2['사고횟수'] >= 1].reset_index(drop=True)
b_3

In [ ]:
# 시각화
m = folium.Map(
    location=[37.1516633,127.0359803],
    zoom_start=12
)


# 오산시 표시
folium.GeoJson(
    df31,
    name='osan',
    style_function=lambda feature: {
        'color': 'grey'}
).add_to(m)


# 20개 우선설치구역 표시
for i in range(len(final_20)):
    folium.vector_layers.Circle([final_20.loc[i,'center_lat'], final_20.loc[i,'center_lon']],radius=300,color='Black',
                               tooltip=final_20.loc[i,'격자명']).add_to(m)

    
# 사고다발지역 표시
for i in range(len(b_3)):
    folium.GeoJson(
        b_3.loc[i,'geometry'],
        name='sago',
        style_function=lambda feature: {
        'color': 'Yellow'},
        tooltip=b_3.loc[i,'격자명']
).add_to(m)

m.save('example.html')
m

In [ ]:
# 격자할당
R = 6373.0

#초기값 생성
count_df5 = b_3[['격자명','lat','lon']]
count_df5['할당격자'] = 0
count_df5['할당_위도'] = 0
count_df5['할당_경도'] = 0
count_df5['시설물'] = '가드레일'


for k in range(len(b_3)):
    buffer = b_3.loc[k,'geometry']
    lat1 = radians(b_3.loc[k,'lat'])
    lon1 = radians(b_3.loc[k,'lon'])
    dist_list = []
    index_list = []
    
    for i in list(final_20[final_20['geometry'].intersection(buffer).is_empty == False].index):
        lat2 = radians(final_20.loc[i,'center_lat'])
        lon2 = radians(final_20.loc[i,'center_lon'])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        dist_list.append(distance)
        index_list.append(i)
   
    if dist_list == []:
        continue
    else:
        close_distance = min(dist_list)
        index_tmp = dist_list.index(close_distance)
        optimal_index = index_list[index_tmp]
        count_df5.loc[k,'할당격자'] = final_20.loc[optimal_index, '격자명']
        count_df5.loc[k,'할당_위도'] = final_20.loc[optimal_index, 'center_lat']
        count_df5.loc[k,'할당_경도'] = final_20.loc[optimal_index, 'center_lon']

        
count_df5

## 시각화6: 과속방지턱
- 초등학교 + 어린이집/유치원 + 학원및교습소(어린이) >= 5
- 도로폭 6m이상 + 왕복2~4차로 or 도로폭 6m미만 + 왕복 2차로

In [ ]:
# 초등학교 개수를 격자별로 Count

df10_a = df10[df10['학교구분'] == '초등학교'].reset_index() # 초등학교 개수만 필터링

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df10_1 = pd.DataFrame({'격자명':list_tmp})
df10_1['초등학교개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df10_a)):
        if (df10_a.loc[j, '학교위치_위도'] >= min_lat) & (df10_a.loc[j, '학교위치_위도'] <= max_lat) & (df10_a.loc[j, '학교위치_경도'] >= min_lon) & (df10_a.loc[j, '학교위치_경도'] <= max_lon):
            df10_1.loc[i,'초등학교개수'] = df10_1.loc[i,'초등학교개수']+1
df10_1

In [ ]:
# 유치원/어린이집개수를 격자별로 Count

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df13_1 = pd.DataFrame({'격자명':list_tmp})
df13_1['유치원/어린이집개수'] = 0


for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df13)):
        if (df13.loc[j, '시설위치_위도'] >= min_lat) & (df13.loc[j, '시설위치_위도'] <= max_lat) & (df13.loc[j, '시설위치_경도'] >= min_lon) & (df13.loc[j, '시설위치_경도'] <= max_lon):
            df13_1.loc[i,'유치원/어린이집개수'] = df13_1.loc[i,'유치원/어린이집개수']+1
df13_1

In [ ]:
# 학원들 중 어린이가 많이 다니는 학원을 필터링

cond1 = df30['교습과정명'].str.contains('초등', na=False)
cond2 = df30['교습과정명'].str.contains('저학년', na=False)
cond3 = df30['교습과정명'].str.contains('초1', na=False)
cond4 = df30['교습과정명'].str.contains('초2', na=False)
cond5 = df30['교습과정명'].str.contains('초3', na=False)
cond6 = df30['교습과정명'].str.contains('초4', na=False)
cond7 = df30['교습과정명'].str.contains('초5', na=False)
cond8 = df30['교습과정명'].str.contains('초6', na=False)

df30_a = df30.loc[cond1 | cond2 | cond3 | cond4 | cond5 | cond6 | cond7 | cond8,:]
df30_a = df30_a.reset_index(drop=True)
df30_a

In [ ]:
# 유치원/어린이집개수를 격자별로 Count

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df30_1 = pd.DataFrame({'격자명':list_tmp})
df30_1['학원교습소개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df30_a)):
        if (df30_a.loc[j, '시설위치_위도'] >= min_lat) & (df30_a.loc[j, '시설위치_위도'] <= max_lat) & (df30_a.loc[j, '시설위치_경도'] >= min_lon) & (df30_a.loc[j, '시설위치_경도'] <= max_lon):
            df30_1.loc[i,'학원교습소개수'] = df30_1.loc[i,'학원교습소개수']+1
df30_1

In [ ]:
# 초등학교, 어린이집/유치원, 학원의 합을 도출한 후, 5개 이상인 지역만 골라낸다.
c_1 = pd.concat([df10_1.iloc[:,[0,1]], df13_1.iloc[:,1], df30_1.iloc[:,1],df2_a[['geometry','lat','lon']]],axis=1)
c_1['sum'] = c_1['초등학교개수'] + c_1['유치원/어린이집개수'] + c_1['학원교습소개수']
c_2 = c_1[c_1['sum'] >= 5].reset_index(drop=True)
c_2

In [ ]:
# geopandas를 활용하여 상세도로망 데이터를 geojson --> csv
df23_10 = gpd.read_file('23.오산시_상세도로망_LV6.geojson')

# multilinestring --> buffer 변환
for i in range(len(df23_10)):
    df23_10.loc[i,'geometry'] = df23_10.loc[i,'geometry'].buffer(0.00005)

df23_10

In [ ]:
# 상행과 하행 차선의 합을 구해 차선의 수를 도출하고, 데이터타입을 object -> int변경
df23_10['lanes'] = 0
for i in range(len(df23_10)):
    df23_10.loc[i,'lanes'] = int(df23_10.loc[i,'up_lanes']) + int(df23_10.loc[i,'dw_lanes'])

c_traffic = df23_10[['link_id','road_rank','geometry','lanes','width']]
c_traffic['road_rank'] = c_traffic['road_rank'].astype('int8')
c_traffic['lanes'] = c_traffic['lanes'].astype('int8')
c_traffic['width'] = c_traffic['width'].astype('int8')
c_traffic

In [ ]:
# 고속국도 및 고속국도 진출지역은 과속방지턱을 설치할 수 없다고 판단하여 제거
# 도로포 및 차선조건 추가

c_traffic = c_traffic[c_traffic['road_rank'].isin(['107','106','103','105'])]
cond1 = ((c_traffic['width'] >= 6) & (c_traffic['lanes'] >= 2) & (c_traffic['lanes'] <= 4))
cond2 = ((c_traffic['width'] < 6) & (c_traffic['lanes'] == 2))
c_traffic = c_traffic[cond1 | cond2].reset_index(drop=True)
c_traffic

In [ ]:
# 시각화
import folium

m = folium.Map(
    location=[37.1516633,127.0359803],
    zoom_start=12
)


# 오산시 표시
folium.GeoJson(
    df31,
    style_function=lambda feature: {
        'color': 'gray'},
    name='osan'
).add_to(m)


# 20개 우선설치구역 표시
for i in range(len(final_20)):
    folium.vector_layers.Circle([final_20.loc[i,'center_lat'], final_20.loc[i,'center_lon']],radius=300,color='Black',
                               tooltip=final_20.loc[i,'격자명']).add_to(m)

    
# 건물 표시
for i in range(len(c_2)):
    folium.GeoJson(
        c_2.loc[i,'geometry'],
        name='building',
        style_function=lambda feature: {
        'color': 'Yellow'},
        tooltip=c_2.loc[i,'격자명']
).add_to(m)
    
# 도로망 표시 (도로폭 및 차선조건)
folium.GeoJson(
    c_traffic,
    name='road',
    style_function=lambda feature: {
        'color': 'Red'
    }
).add_to(m)    
    
m.save('example.html')
m

In [ ]:
# 격자할당
R = 6373.0

# 우선적으로 교통량이 많은 지역과 건물이 밀집한 지역을 선택
possible_list = []
for k in range(len(c_traffic)):
    buffer = c_traffic['geometry'][k].buffer(0.0001)
        
    for i in list(c_2[gpd.GeoDataFrame(c_2['geometry'])['geometry'].intersection(buffer).is_empty == False].index):
        possible_list.append(i)
        
possible_list = list(set(possible_list))
c_3 = c_2.iloc[possible_list,:].reset_index(drop=True)

#초기값 생성
count_df6 = c_3[['격자명','lat','lon']]
count_df6['할당격자'] = 0
count_df6['할당_위도'] = 0
count_df6['할당_경도'] = 0
count_df6['시설물'] = '과속방지턱'


for k in range(len(c_3)):
    buffer = c_3.loc[k,'geometry']
    lat1 = radians(c_3.loc[k,'lat'])
    lon1 = radians(c_3.loc[k,'lon'])
    dist_list = []
    index_list = []
    
    for i in list(final_20[final_20['geometry'].intersection(buffer).is_empty == False].index):
        lat2 = radians(final_20.loc[i,'center_lat'])
        lon2 = radians(final_20.loc[i,'center_lon'])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        dist_list.append(distance)
        index_list.append(i)
   
    if dist_list == []:
        continue
    else:
        close_distance = min(dist_list)
        index_tmp = dist_list.index(close_distance)
        optimal_index = index_list[index_tmp]
        count_df6.loc[k,'할당격자'] = final_20.loc[optimal_index, '격자명']
        count_df6.loc[k,'할당_위도'] = final_20.loc[optimal_index, 'center_lat']
        count_df6.loc[k,'할당_경도'] = final_20.loc[optimal_index, 'center_lon']

        
count_df6

### 시각화7: 단속카메라
- 사고횟수 4회이상 Count

In [ ]:
# TAAS 사고횟수 데이터를 격자별로 할당

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

d_1 = pd.DataFrame({'격자명':list_tmp})
d_1['사고횟수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df)):
        if (df.loc[j, 'lat'] >= min_lat) & (df.loc[j, 'lat'] <= max_lat) & (df.loc[j, 'lon'] >= min_lon) & (df.loc[j, 'lon'] <= max_lon):
            d_1.loc[i,'사고횟수'] += 1
            

d_1 = pd.concat([d_1, df2_a[['geometry','lat','lon']]], axis=1)
d_1

In [ ]:
# 사고횟수가 4회이상인 것을 필터링
d_2 = d_1[d_1['사고횟수'] >= 4].reset_index(drop=True)
d_2

In [ ]:
# 시각화
m = folium.Map(
    location=[37.1516633,127.0359803],
    zoom_start=12
)


# 오산시 표시
folium.GeoJson(
    df31,
    style_function=lambda feature: {
        'color': 'gray'},
    name='osan'
).add_to(m)


# 20개 우선설치구역 표시
for i in range(len(final_20)):
    folium.vector_layers.Circle([final_20.loc[i,'center_lat'], final_20.loc[i,'center_lon']],radius=300,color='Black',
                               tooltip=final_20.loc[i,'격자명']).add_to(m)

    
# 사고가 일어난 곳 표시
for i in range(len(d_2)):
    folium.GeoJson(
        d_2.loc[i,'geometry'],
        name='sago',
        style_function=lambda feature: {
        'color': 'Yellow'},
        tooltip=d_2.loc[i,'격자명']
).add_to(m)
    
    
m.save('example.html')
m

In [ ]:
# 격자할당
R = 6373.0

#초기값 생성
count_df7 = d_2[['격자명','lat','lon']]
count_df7['할당격자'] = 0
count_df7['할당_위도'] = 0
count_df7['할당_경도'] = 0
count_df7['시설물'] = '단속카메라'


for k in range(len(d_2)):
    buffer = d_2.loc[k,'geometry']
    lat1 = radians(d_2.loc[k,'lat'])
    lon1 = radians(d_2.loc[k,'lon'])
    dist_list = []
    index_list = []
    
    for i in list(final_20[final_20['geometry'].intersection(buffer).is_empty == False].index):
        lat2 = radians(final_20.loc[i,'center_lat'])
        lon2 = radians(final_20.loc[i,'center_lon'])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        dist_list.append(distance)
        index_list.append(i)
   
    if dist_list == []:
        continue
    else:
        close_distance = min(dist_list)
        index_tmp = dist_list.index(close_distance)
        optimal_index = index_list[index_tmp]
        count_df7.loc[k,'할당격자'] = final_20.loc[optimal_index, '격자명']
        count_df7.loc[k,'할당_위도'] = final_20.loc[optimal_index, 'center_lat']
        count_df7.loc[k,'할당_경도'] = final_20.loc[optimal_index, 'center_lon']


count_df7

### 우선설치구역의 필요시설물소요 종합
- 할당격자: 우선설치지역의 위험순위
- 할당_위도, 할당_경도: 해당 우선설치지역의 중심좌표
- 격자명: 우선설치지역 내 시설물이 설치되는 격자의 고유이름
- lat, lon: 우선설치지역 내 시설물이 설치되는 격자의 좌표(=시설물 설치좌표)
- 시설물: 설치할 시설물

In [ ]:
# 최종 필요시설물 Concat

count_df = pd.concat([count_df1, count_df2, count_df3, count_df4, count_df5, count_df6, count_df7])
count_df = count_df[['할당격자','할당_위도','할당_경도','격자명','lat','lon','시설물']]
count_df[count_df['할당격자'] > 0].sort_values(by='할당격자')